In [38]:
import pickle
import networkx as nx
import pandas as pd
import numpy as np

In [39]:
individualMetricsBarcelona = "../Data/Barcelona/04Stage/Metrics/Individual/normalizateIndividualnetworkMetrics.pkl"
metadataPath = "../Data/Barcelona/03Stage/finalMetadataBarcelona.csv"
playersPath =  "../Data/Barcelona/04Stage/playersList.pkl"  



In [40]:
def loadMetadataFile(path):
    try:
        # Open the CSV file
        # Create a CSV reader object
        dfScore = pd.read_csv(path)
        return dfScore
    except FileNotFoundError:
        print("The file does not exist.")
    except Exception as e:
        print("An error occurred:", e)

In [41]:
def load_metrics(file):
    try:
        with open(file, "rb") as f:
            deserializedFile = pickle.load(f)
        return deserializedFile
    except FileNotFoundError:
        print(f"File '{individualMetricsBarcelona}' not found.")
    except nx.NetworkXError as e:
        print(f"Error reading graph from '{individualMetricsBarcelona}': {e}")

In [42]:
def calculateMetrics(data):
    dfNoNan = data.fillna(" ")
    dfNoBlank = dfNoNan.replace(" ", float('NaN'))  # Replace blank spaces with 0
    # Calculate mean, standard deviation, and count
    meanValues = dfNoBlank.mean(axis=1).round(2)
    stdValues = dfNoBlank.std(axis=1).round(2)
    countValues = dfNoBlank.count(axis=1)

    return meanValues, stdValues, countValues

In [43]:
def nomralizatedMetrics(file):
    finalDict = {}
    for element in file:
        #print(element)
        concatenated_dict = {}
        for score in file[element]:
            #print(score)
            metricsTable = pd.DataFrame()
            df = pd.DataFrame.from_dict(file[element][score], orient="index")
            #print(df)
            meanValues, stdValues, countValues = calculateMetrics(df)  
            a = meanValues.dropna(axis=0)
            if  not a.empty:
                classifyValues = pd.cut(np.array(a), 5, labels=["worst", "bad", "medium", "good", "excellent"]).astype(str)
                columnClassifiy = pd.DataFrame({'Class': classifyValues}, index=a.index)
                metricsTable = pd.concat([metricsTable, a.rename('Mean'), stdValues.rename('Std'), countValues.rename('Count'), columnClassifiy], axis=1)
                #print(metricsTable)
                metricsTable.set_index(df.index, inplace=True)
                #print(metricsTable)
            # print(dfN)
                b = metricsTable.T.to_dict(orient="list")
                #print(b)
                concatenated_dict[score] = b
        finalDict[element] = concatenated_dict
    return finalDict

In [44]:
file = load_metrics(individualMetricsBarcelona)

In [45]:
finalDict = nomralizatedMetrics(file)

In [46]:
for element in finalDict:
    print(finalDict[element])

{'0_1': {'6616': [0.4, 0.38, 2, 'bad'], '5503': [0.39, 0.26, 5, 'bad'], '5203': [0.64, 0.31, 8, 'medium'], '6379': [0.57, 0.21, 7, 'medium'], '5506': [0.55, 0.23, 7, 'medium'], '4691': [0.12, 0.17, 7, 'worst'], '4324': [0.33, 0.21, 5, 'bad'], '4320': [0.63, 0.34, 8, 'medium'], '5213': [0.58, 0.35, 8, 'medium'], '11094': [0.35, 0.03, 2, 'bad'], '6998': [0.46, 0.29, 2, 'bad'], '5470': [0.53, 0.21, 6, 'medium'], '5211': [0.6, 0.31, 7, 'medium'], '5216': [0.56, 0.43, 4, 'medium'], '5246': [0.65, 0.26, 8, 'good'], '20055': [0.31, 0.44, 2, 'bad'], '6332': [1.0, nan, 1, 'excellent'], '10609': [0.37, 0.21, 5, 'bad'], '3508': [0.72, 0.39, 2, 'good'], '7068': [0.62, nan, 1, 'medium'], '6400': [0.88, nan, 1, 'excellent']}, '0_2': {'5246': [0.5, 0.17, 3, 'medium'], '5503': [1.0, nan, 1, 'excellent'], '6379': [0.31, 0.34, 3, 'bad'], '5213': [0.54, 0.12, 3, 'medium'], '5506': [0.71, 0.06, 2, 'good'], '5211': [0.47, 0.04, 2, 'medium'], '5203': [0.8, 0.26, 3, 'good'], '5216': [0.56, nan, 1, 'medium'],

In [47]:
metadataFile =  loadMetadataFile(metadataPath)

In [55]:
for score in metadataFile["Score"].unique():
    print(score)
    metricsTable = pd.DataFrame()
    element_values = []
    for element in finalDict:
        if score in finalDict[element]:
            if not element in element_values:
                element_values.append(element)
            df = pd.DataFrame.from_dict(finalDict[element][score], orient='index')
            metricsTable = pd.concat([metricsTable, df], axis=1)
    multi_index = pd.MultiIndex.from_product([element_values, ['Mean', 'Std', 'Count', 'Class']], names=[None, None])
    metricsTable.columns = multi_index
    #metricsTable.to_csv(f"{targetPath}/Score/Individual/{score}_individualMetrics.csv", index=True)
    metricsTable.fillna(0, inplace=True)
    print(metricsTable)
    

0_1
        inD                         outD                        clust        \
       Mean   Std Count      Class  Mean   Std Count      Class  Mean   Std   
6616   0.40  0.38     2        bad  0.29  0.00     2        bad  0.72  0.21   
5503   0.39  0.26     5        bad  0.31  0.35     5        bad  0.52  0.21   
5203   0.64  0.31     8     medium  0.60  0.36     9       good  0.54  0.26   
6379   0.57  0.21     7     medium  0.60  0.17     7       good  0.73  0.23   
5506   0.55  0.23     7     medium  0.47  0.44     7     medium  0.55  0.27   
4691   0.12  0.17     7      worst  0.21  0.18     7      worst  0.80  0.19   
4324   0.33  0.21     5        bad  0.41  0.27     5        bad  0.10  0.23   
4320   0.63  0.34     8     medium  0.61  0.37     9       good  0.42  0.35   
5213   0.58  0.35     8     medium  0.65  0.31     8       good  0.42  0.34   
11094  0.35  0.03     2        bad  0.21  0.10     2      worst  0.76  0.34   
6998   0.46  0.29     2        bad  0.29  0.40  

In [49]:
playersList = load_metrics(playersPath)

In [54]:
element_values = []
scoreMetrics = {}

for key in playersList.keys():
    print(key)
    previousMetricsTable = pd.DataFrame()
    metricsTable = pd.DataFrame()
    for element in finalDict:
        if not element in element_values:
            element_values.append(element)
        for score in finalDict[element]:
            if score in finalDict[element]:
                allValues = []
                if str(key) in finalDict[element][score].keys():
                    values = finalDict[element][score][str(key)]
                    if values:           
                        # Create or update dictionary entry for the score
                        if score not in scoreMetrics:
                            scoreMetrics[score] = {'Mean': values[0], 'Std': values[1], 'Count': values[2], 'Class': values[3]}
                        else:
                            scoreMetrics[score]['Mean'] = values[0]
                            scoreMetrics[score]['Std'] = values[1]
                            scoreMetrics[score]['Count'] = values[2]
                            scoreMetrics[score]['Class'] = values[3]
                    else:
                        if score not in scoreMetrics:
                            scoreMetrics[score] = {'Mean': 0, 'Std': 0, 'Count': 0, 'Class':0}
                        else:
                            scoreMetrics[score]['Mean'] = 0
                            scoreMetrics[score]['Std'] = 0
                            scoreMetrics[score]['Count'] = 0
                            scoreMetrics[score]['Class'] = 0
                else:
                    if score not in scoreMetrics:
                        scoreMetrics[score] = {'Mean': 0, 'Std': 0, 'Count': 0, 'Class': 0}
                    else:
                        scoreMetrics[score]['Mean'] = 0
                        scoreMetrics[score]['Std'] = 0
                        scoreMetrics[score]['Count'] = 0
                        scoreMetrics[score]['Class'] = 0

                    #df = pd.DataFrame.from_dict(finalDict[element][score][str(key)], orient='index')
                    #metricsTable = pd.concat([metricsTable, df], axis=1)
                    #print(metricsTable)
        previousMetricsTable = pd.DataFrame.from_dict(scoreMetrics, orient='index')
        metricsTable = pd.concat([metricsTable, previousMetricsTable], axis = 1) 
    metricsTable.columns.name = None
    multi_index = pd.MultiIndex.from_product([element_values, ['Mean', 'Std', 'Count','Class']], names=[None, None])
    metricsTable.columns = multi_index
    #metricsTable.to_csv(f"{targetPath}/Player/{key}_individualMetrics.csv", index = True)
    metricsTable.fillna(0, inplace=True)
    print(metricsTable)

  




4320
      inD                         outD                     clust        ...  \
     Mean   Std Count      Class  Mean   Std Count   Class  Mean   Std  ...   
0_1  0.63  0.34     8     medium  0.61  0.37     9    good  0.42  0.35  ...   
0_2  0.45  0.21     3     medium  0.36  0.43     3     bad  0.25  0.43  ...   
1_2  0.75  0.25     3  excellent  0.49  0.44     3  medium  0.57  0.31  ...   
1_0  0.66  0.31    24  excellent  0.56  0.29    24    good  0.47  0.35  ...   
2_0  0.57  0.30    17       good  0.50  0.34    17  medium  0.65  0.32  ...   
3_0  0.56  0.35    13       good  0.43  0.33    13    good  0.35  0.28  ...   
4_0  0.62  0.30     9       good  0.46  0.29     9    good  0.54  0.30  ...   
5_0  0.58  0.44     4     medium  0.58  0.13     4    good  0.52  0.38  ...   
6_0  0.42  0.29     3     medium  0.67  0.29     3    good  0.37  0.32  ...   
7_0  1.00  0.00     1  excellent  0.25  0.00     1     bad  0.54  0.00  ...   
8_0  1.00  0.00     1  excellent  0.00  0.00   

In [51]:
"""     
            # Concatenate horizontally with the main DataFrame
        previousMetricsTable = pd.DataFrame.from_dict(scoreMetrics, orient='index')
        #classifyValues = pd.cut(np.array(previousMetricsTable["Mean"]), 5, labels=["worst", "bad", "medium", "good", "excellent"]).astype(str)
        #columnClassifiy = pd.DataFrame({'Class': classifyValues}, index=previousMetricsTable["Mean"].index)
        metricsTable = pd.concat([metricsTable, previousMetricsTable], axis = 1)                                      
        
    metricsTable.columns.name = None
    multi_index = pd.MultiIndex.from_product([element_values, ['Mean', 'Std', 'Count']], names=[None, None])
    metricsTable.columns = multi_index
    #metricsTable.to_csv(f"{targetPath}/Player/{key}_individualMetrics.csv", index = True)
    metricsTable.fillna(0, inplace=True)
    metricsTable.to_pickle(f"{targetPath}/Player/{key}_individualMetrics.pkl"
"""

'     \n            # Concatenate horizontally with the main DataFrame\n        previousMetricsTable = pd.DataFrame.from_dict(scoreMetrics, orient=\'index\')\n        #classifyValues = pd.cut(np.array(previousMetricsTable["Mean"]), 5, labels=["worst", "bad", "medium", "good", "excellent"]).astype(str)\n        #columnClassifiy = pd.DataFrame({\'Class\': classifyValues}, index=previousMetricsTable["Mean"].index)\n        metricsTable = pd.concat([metricsTable, previousMetricsTable], axis = 1)                                      \n        \n    metricsTable.columns.name = None\n    multi_index = pd.MultiIndex.from_product([element_values, [\'Mean\', \'Std\', \'Count\']], names=[None, None])\n    metricsTable.columns = multi_index\n    #metricsTable.to_csv(f"{targetPath}/Player/{key}_individualMetrics.csv", index = True)\n    metricsTable.fillna(0, inplace=True)\n    metricsTable.to_pickle(f"{targetPath}/Player/{key}_individualMetrics.pkl"\n'